# OTT 서비스 추천(장르 가중치 방식)

In [1]:
import pandas as pd
import numpy as np
import re
import urllib.request
import warnings
warnings.filterwarnings('ignore')

# 데이터 전처리

In [2]:
#데이터 다운로드
urllib.request.urlretrieve("https://raw.githubusercontent.com/world970511/elice_OTT/master/dataset/kinolight_all_data.csv",
                        filename="OTT.csv")
data = pd.read_csv('OTT.csv',encoding='utf-8')
data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021.0,TV,15세이상관람가,"['스릴러', 'SF']",['한국'],"['배두나', '공유', '이준', '김선영', '이무생', '허성태', '이성욱']",['최항용'],NaN,['넷플릭스']
1,돈 룩 업(Don't Look Up),2021.0,movie,15세이상관람가,['코미디'],['미국'],"['레오나르도 디카프리오 ', '제니퍼 로렌스', '롭 모건', '조나 힐', '마...",['아담 맥케이'],139.0,['넷플릭스']
2,설강화(Snowdrop),2021.0,TV,15세이상관람가,['드라마'],['한국'],"['정해인', '지수', '유인나', '장승조', '윤세아', '김혜윤', '정유진']",['조현탁'],NaN,['디즈니+']
3,나 홀로 집에(Home Alone),1990.0,movie,전체관람가,"['코미디', '어드벤처(모험)', '가족']",['미국'],"['맥컬리 컬킨 ', '조 페시', '다니엘 스턴 ', '존 허드', '로버츠 블로...",['크리스 콜럼버스'],105.0,['디즈니+']
4,매트릭스(The Matrix),1999.0,movie,12세이상관람가,"['액션', 'SF']",['미국'],"['키아누 리브스 ', '로렌스 피시번 ', '캐리 앤 모스 ', '휴고 위빙', ...","['릴리 워쇼스키', '라나 워쇼스키']",136.0,"['웨이브', '티빙', '왓챠', '쿠팡플레이']"


In [3]:
platform = ['넷플릭스', '웨이브', '티빙', '쿠팡플레이', '왓챠', '디즈니+', 'seezn', '라프텔']

In [4]:
data.loc[997, 'year'] = '2021.0'
data.loc[2114, 'year'] = '2019.0'
data.loc[2294, 'year'] = '2019.0'
data.loc[15295, 'year'] = '2011.0'
data.loc[16806, 'year'] = '2020.0'
data.loc[19272, 'year'] = '2013.0'
data.loc[23503, 'year'] = '2015.0'
data.loc[23539, 'year'] = '2019.0'
data.loc[23592, 'year'] = '2014.0'
data.loc[23598, 'year'] = '2015.0'
data.loc[23799, 'year'] = '2012.0'
data.loc[23832, 'year'] = '2018.0'
data.loc[24536, 'year'] = '2019.0'
data.loc[24539, 'year'] = '2019.0'
data.loc[24970, 'year'] = '2020.0'
data.loc[25250, 'year'] = '2016.0'
data.loc[25251, 'year'] = '2015.0'
data.loc[26295, 'year'] = '2021.0'

In [5]:
#중복제거 및 기타 제거
li=['genre','country','cast','director','provider',]

for i in li:
  data[i]=data[i].apply(lambda x:re.sub(r'\([^)]*\)', '',x.replace('[','').replace(']','').replace("'",'').replace("'",'')))
  data[i]=data[i].apply(lambda x:re.sub(r' ', '',x))
  data[i]=data[i].apply(lambda x: ' '.join(set(x.strip().split(','))))

data.head(5)

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021.0,TV,15세이상관람가,SF 스릴러,한국,허성태 이성욱 이준 이무생 공유 배두나 김선영,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021.0,movie,15세이상관람가,코미디,미국,레오나르도디카프리오 티모시샬라메 마크라이런스 롭모건 조나힐 제니퍼로렌스 타일러페리,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021.0,TV,15세이상관람가,드라마,한국,정유진 지수 김혜윤 장승조 정해인 윤세아 유인나,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990.0,movie,전체관람가,가족 어드벤처 코미디,미국,다니엘스턴 조페시 존허드 캐서린오하라 로버츠블로썸 맥컬리컬킨 안젤라고덜즈,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999.0,movie,12세이상관람가,SF 액션,미국,로렌스피시번 조판토리아노 키아누리브스 휴고위빙 글로리아포스터 캐리앤모스,릴리워쇼스키 라나워쇼스키,136.0,티빙 왓챠 쿠팡플레이 웨이브


In [6]:
# 불필요한 행 제거
data= data[data['title']!='title']

# 년도 정수형으로 변경
data['year']=data['year'].astype('float').astype('Int64')

# 제작미정,시사/교양,기타(장르가 아니라 country로 가있음;;) 정리
# 제작미정인 경우 풀릴 예정인 곳과 아닌 곳이 있고 양이 너무 많아서 그냥 장르로 변경함
def r_ch(s,i):
  li=s.split(',')
  if i in li: 
    li.remove(i)
  return ','.join (li)

def add_ch(s,i):
  li=s.split(' ')
  if i not in li: li+=[i]
  return ' '.join (li)

li=['미정','시사/교양','기타']
for i in li:
    d=data.loc[data["country"].str.contains(i)]
    data=data.loc[data["country"].str.contains(i)==False]

    d['country']=d['country'].apply(lambda x:r_ch(x,i))
    d['genre']=d['genre'].apply(lambda x:add_ch(x,i))

    data=pd.concat([data,d])

data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,SF 스릴러,한국,허성태 이성욱 이준 이무생 공유 배두나 김선영,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,미국,레오나르도디카프리오 티모시샬라메 마크라이런스 롭모건 조나힐 제니퍼로렌스 타일러페리,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,정유진 지수 김혜윤 장승조 정해인 윤세아 유인나,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,가족 어드벤처 코미디,미국,다니엘스턴 조페시 존허드 캐서린오하라 로버츠블로썸 맥컬리컬킨 안젤라고덜즈,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999,movie,12세이상관람가,SF 액션,미국,로렌스피시번 조판토리아노 키아누리브스 휴고위빙 글로리아포스터 캐리앤모스,릴리워쇼스키 라나워쇼스키,136.0,티빙 왓챠 쿠팡플레이 웨이브


In [7]:
data.kind = data.kind.astype('string')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27301 entries, 0 to 23686
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   title         27301 non-null  object
 1   year          27301 non-null  Int64 
 2   kind          27301 non-null  string
 3   KMRB          26082 non-null  object
 4   genre         27301 non-null  object
 5   country       27301 non-null  object
 6   cast          27301 non-null  object
 7   director      27301 non-null  object
 8   runtime(min)  16866 non-null  object
 9   provider      27301 non-null  object
dtypes: Int64(1), object(8), string(1)
memory usage: 2.3+ MB


In [8]:
#독점 및 오리지널 표시 및 필요한 provider데이터 외에 드랍

##필요한 provider외에 나머지 드랍
remove_provider_list=['네이버시리즈온', 'GooglePlay무비', '씨네폭스','아마존프라임비디오','AppleTV+']
n_data=pd.DataFrame(columns=['title',	'year',	'kind',	'KMRB',	'genre',	'country',	'cast',	'director',	'runtime(min)',	'provider'])
for i in remove_provider_list:
  data['provider']=data['provider'].apply(lambda x: x.replace(i,''))

data['provider'].dropna()

data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,SF 스릴러,한국,허성태 이성욱 이준 이무생 공유 배두나 김선영,최항용,NaN,넷플릭스
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,미국,레오나르도디카프리오 티모시샬라메 마크라이런스 롭모건 조나힐 제니퍼로렌스 타일러페리,아담맥케이,139.0,넷플릭스
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,한국,정유진 지수 김혜윤 장승조 정해인 윤세아 유인나,조현탁,NaN,디즈니+
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,가족 어드벤처 코미디,미국,다니엘스턴 조페시 존허드 캐서린오하라 로버츠블로썸 맥컬리컬킨 안젤라고덜즈,크리스콜럼버스,105.0,디즈니+
4,매트릭스(The Matrix),1999,movie,12세이상관람가,SF 액션,미국,로렌스피시번 조판토리아노 키아누리브스 휴고위빙 글로리아포스터 캐리앤모스,릴리워쇼스키 라나워쇼스키,136.0,티빙 왓챠 쿠팡플레이 웨이브


In [9]:
# 장르, 국가, 제공 플랫폼 리스트로 변경
data['genre']=data['genre'].apply(lambda x: [i for i in x.split(' ')])
data['provider']=data['provider'].apply(lambda x: [i for i in x.split(' ')])
data['country']=data['country'].apply(lambda x: [i for i in x.split(' ')])

# 독점/오리지널 여부
data.loc[:,'origin']=data['provider'].apply(lambda x: True if len(x)==1 else False)
data = data.sort_index()

In [10]:
data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,origin
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,"[SF, 스릴러]",[한국],허성태 이성욱 이준 이무생 공유 배두나 김선영,최항용,NaN,[넷플릭스],True
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,[코미디],[미국],레오나르도디카프리오 티모시샬라메 마크라이런스 롭모건 조나힐 제니퍼로렌스 타일러페리,아담맥케이,139.0,[넷플릭스],True
2,설강화(Snowdrop),2021,TV,15세이상관람가,[드라마],[한국],정유진 지수 김혜윤 장승조 정해인 윤세아 유인나,조현탁,NaN,[디즈니+],True
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,"[가족, 어드벤처, 코미디]",[미국],다니엘스턴 조페시 존허드 캐서린오하라 로버츠블로썸 맥컬리컬킨 안젤라고덜즈,크리스콜럼버스,105.0,[디즈니+],True
4,매트릭스(The Matrix),1999,movie,12세이상관람가,"[SF, 액션]",[미국],로렌스피시번 조판토리아노 키아누리브스 휴고위빙 글로리아포스터 캐리앤모스,릴리워쇼스키 라나워쇼스키,136.0,"[티빙, 왓챠, 쿠팡플레이, 웨이브]",False


In [11]:
for i, j in zip(data.genre, data.provider):
  if '라프텔' in j and '애니메이션' not in i:
    i.append('애니메이션')

In [12]:
# 상위 10개의 작품을 선호한다고 했을 때 장르 리스트 생성 및 언급 횟수에 따른 가중치 딕셔너리 생성
genre_pop = [x for x in data.iloc[:10].genre]
genre_list = []
for i in genre_pop:
  for j in i:
    genre_list.append(j)

genre_rank = {}
for i in genre_list:
  genre_rank[i] = genre_list.count(i)
genre_rank

{'SF': 2,
 '가족': 1,
 '드라마': 3,
 '멜로/로맨스': 1,
 '스릴러': 2,
 '애니메이션': 1,
 '액션': 2,
 '어드벤처': 3,
 '코미디': 3}

In [13]:
# 상위 10개의 작품을 선호한다고 했을 때 장르 리스트 생성 및 언급 횟수에 따른 가중치 딕셔너리 생성
country_pop = [x for x in data.iloc[:10].country]
country_list = []
for i in country_pop:
  for j in i:
    country_list.append(j)

country_rank = {}
for i in country_list:
  country_rank[i] = country_list.count(i)
best_country = {}
for key, val in country_rank.items():
  if val == max(country_rank.values()):
    best_country[key] = val
best_country = list(best_country.keys())
best_country

['한국', '미국']

In [14]:
# 장르에 따른 포인트 계산 후 데이터 프레임에 추가
rank_point = []
# a = {'코미디':3, '가족':3, '드라마':4, '스릴러':7}
# b = ['한국']
# g = []

for contents_genre in data.genre:
  cnt = 0
  for cg in contents_genre:
    if cg in list(genre_rank.keys()):
      cnt += genre_rank[cg]
  cnt -= len(contents_genre)
  rank_point.append(cnt)

for i in range(len(data.country)):
  for cc in data.country.iloc[i]:
    if cc in best_country:
      rank_point[i] += 3

data['genre_point'] = rank_point

In [15]:
# 높은 순으로 정렬된 데이터 생성, 최대 포인트-4 이상의 포인트를 가진 컨텐츠만 추출
ranking = data.sort_values('genre_point', ascending=False)
ranking = ranking[ranking['genre_point']>=max(rank_point)//2+2]

In [16]:
# from google.colab import drive
# drive.mount('/content/drive')

In [17]:
# 컨텐츠 인기순으로 재배열
ranking = ranking.sort_index()
ranking.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,origin,genre_point
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,"[가족, 어드벤처, 코미디]",[미국],다니엘스턴 조페시 존허드 캐서린오하라 로버츠블로썸 맥컬리컬킨 안젤라고덜즈,크리스콜럼버스,105.0,[디즈니+],True,7
6,엔칸토: 마법의 세계(Encanto),2021,movie,전체관람가,"[애니메이션, 어드벤처, 코미디]","[콜롬비아, 미국]",윌머발더라마 앤지세페다 다이앤게레로 존레귀자모 스테파니비트리즈 제시카대로우,바이론하워드 재러드부시,109.0,[디즈니+],True,7
8,스파이더맨(Spider-Man),2002,movie,12세이상관람가,"[스릴러, 어드벤처, 액션]",[미국],로즈마리해리스 J.K.시몬스 클리프로버트슨 토비맥과이어 커스틴던스트 윌렘대포 제임스프랭코,샘레이미,121.0,"[쿠팡플레이, seezn, 왓챠, 웨이브, 넷플릭스]",False,7
11,스파이더맨: 파 프롬 홈(Spider-Man: Far From Home),2019,movie,12세이상관람가,"[어드벤처, SF, 액션, 코미디]",[미국],제이크질렌할 존파브로 코비스멀더스 젠데이아콜먼 마리사토메이 사무엘L.잭슨 톰홀랜드,존왓츠,129.0,"[왓챠, 넷플릭스, seezn]",False,9
13,인셉션(Inception),2010,movie,12세이상관람가,"[어드벤처, 드라마, 스릴러, SF]",[미국],레오나르도디카프리오 톰하디 엘리엇페이지 와타나베켄 조셉고든레빗 킬리언머피 마리옹꼬띠아르,크리스토퍼놀란,147.0,"[쿠팡플레이, 왓챠, 티빙, 웨이브, 넷플릭스]",False,9


In [18]:
# ranking.info()

In [19]:
# 가장 많은 컨텐츠를 제공하는 플랫폼 순위
platform_pop = [x for x in ranking.provider.iloc[:10]]
platform_list = []
for i in platform_pop:
  for j in i:
    platform_list.append(j)

rank = {}
for i in platform:
  rank[i] = platform_list.count(i)
rank = sorted(rank.items(), key=lambda x:x[1], reverse=True)
print('1등 플랫폼: {}, 2등 플랫폼: {}, 3등 플랫폼: {}'.format(rank[0][0], rank[1][0], rank[2][0]))

1등 플랫폼: 넷플릭스, 2등 플랫폼: 왓챠, 3등 플랫폼: 디즈니+


In [20]:
# 가장 많은 독점/오리지널 컨텐츠를 제공하는 플랫폼 순위
original = []
for i in ranking.provider.iloc[:10]:
  if len(i) == 1:
    original.append(i[0])

original_rank = {}
for i in platform:
  original_rank[i] = original.count(i)
original_rank = sorted(original_rank.items(), key=lambda x:x[1], reverse=True)
print('1등 플랫폼: {}, 2등 플랫폼: {}, 3등 플랫폼: {}'.format(original_rank[0][0], original_rank[1][0], original_rank[2][0]))

1등 플랫폼: 디즈니+, 2등 플랫폼: 넷플릭스, 3등 플랫폼: 웨이브


# 각 플랫폼별 특징 통계

In [21]:
platform = ['넷플릭스', '웨이브', '티빙', '쿠팡플레이', '왓챠', '디즈니+', 'seezn', '라프텔']

In [22]:
# 독점/오리지널 컨텐츠 비율
provider_count = {}
original_count = {}
# 각 플랫폼 당 보유 컨텐츠 개수
for pf in platform:
  cnt = 0
  ogcnt = 0
  for pv, og in zip(data.provider, data.origin):
    if pf in pv:
      cnt += 1
    if (og == True) and pf in pv:
      ogcnt += 1 
  provider_count[pf] = cnt
  original_count[pf] = ogcnt

platforms = pd.DataFrame({'all':provider_count.values(), 'origin':original_count.values()}, index=platform)
platforms['common'] = platforms['all'] - platforms['origin']
platforms

,all,origin,common
넷플릭스,6443,4549,1894
웨이브,10421,2180,8241
티빙,9653,2194,7459
쿠팡플레이,1398,163,1235
왓챠,13876,3715,10161
디즈니+,1601,1477,124
seezn,1752,383,1369
라프텔,1701,320,1381


In [23]:
# 컨텐츠 특징 시각화
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.offline as py
import plotly.express as px
import seaborn as sns

In [24]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "pie"}, {"type": "pie"}]])

fig.add_trace(go.Pie(
     values=platforms['all'],
     labels=platform,
     domain=dict(x=[0, 0.5]),
     name="All platform"), 
     row=1, col=1)

fig.add_trace(go.Pie(
    values=platforms['origin'],
    labels=platform,
    domain=dict(x=[0.5, 1.0]),
    name="Origin Platform"),
    row=1, col=2)

fig.show()

In [25]:
specs = [[{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}], [{'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}, {'type': 'pie'}]]
fig2 = make_subplots(rows=2, cols=4, specs=specs)

fig2.add_trace(go.Pie(
     values=platforms.loc['넷플릭스'],
     labels=platforms.columns,
     name="넷플릭스"), 
     row=1, col=1)

fig2.add_trace(go.Pie(
    values=platforms.loc['웨이브'],
    labels=platforms.columns,
    name="웨이브"),
    row=1, col=2)

fig2.add_trace(go.Pie(
    values=platforms.loc['티빙'],
    labels=platforms.columns,
    name="티빙"),
    row=1, col=3)

fig2.add_trace(go.Pie(
    values=platforms.loc['쿠팡플레이'],
    labels=platforms.columns,
    name="쿠팡플레이"),
    row=1, col=4)

fig2.add_trace(go.Pie(
    values=platforms.loc['왓챠'],
    labels=platforms.columns,
    name="왓챠"),
    row=2, col=1)

fig2.add_trace(go.Pie(
    values=platforms.loc['디즈니+'],
    labels=platforms.columns,
    name="디즈니+"),
    row=2, col=2)

fig2.add_trace(go.Pie(
    values=platforms.loc['라프텔'],
    labels=platforms.columns,
    name="라프텔"),
    row=2, col=3)

fig2.add_trace(go.Pie(
    values=platforms.loc['seezn'],
    labels=platforms.columns,
    name="seezn"),
    row=2, col=4)

fig2.show()

In [26]:
# 플랫폼 별 영화, TV 프로그램 비율
movie_count = {}
tv_count = {}
# 각 플랫폼 당 보유 컨텐츠 개수
for pf in platform:
  mo = 0
  tv = 0
  for ki, pv in zip(data.kind, data.provider):
    if (ki == 'movie') and pf in pv:
      mo += 1
    if (ki == 'TV') and pf in pv:
      tv += 1 
  movie_count[pf] = mo
  tv_count[pf] = tv

kinds = pd.DataFrame({'movie':movie_count.values(), 'tv':tv_count.values()}, index=platform)
kinds

,movie,tv
넷플릭스,3168,3275
웨이브,6283,4138
티빙,6214,3439
쿠팡플레이,803,595
왓챠,9754,4122
디즈니+,881,720
seezn,1335,417
라프텔,416,1285


In [27]:
# 플랫폼 별로 구분된 데이터를 만들기 위한 플랫폼 컬럼 문자열 변환
data['provider']=data['provider'].apply(lambda x: ', '.join(x))
data['genre']=data['genre'].apply(lambda x: ', '.join(x))
data.head()

,title,year,kind,KMRB,genre,country,cast,director,runtime(min),provider,origin,genre_point
0,고요의 바다(The Silent Sea),2021,TV,15세이상관람가,"SF, 스릴러",[한국],허성태 이성욱 이준 이무생 공유 배두나 김선영,최항용,NaN,넷플릭스,True,5
1,돈 룩 업(Don't Look Up),2021,movie,15세이상관람가,코미디,[미국],레오나르도디카프리오 티모시샬라메 마크라이런스 롭모건 조나힐 제니퍼로렌스 타일러페리,아담맥케이,139.0,넷플릭스,True,5
2,설강화(Snowdrop),2021,TV,15세이상관람가,드라마,[한국],정유진 지수 김혜윤 장승조 정해인 윤세아 유인나,조현탁,NaN,디즈니+,True,5
3,나 홀로 집에(Home Alone),1990,movie,전체관람가,"가족, 어드벤처, 코미디",[미국],다니엘스턴 조페시 존허드 캐서린오하라 로버츠블로썸 맥컬리컬킨 안젤라고덜즈,크리스콜럼버스,105.0,디즈니+,True,7
4,매트릭스(The Matrix),1999,movie,12세이상관람가,"SF, 액션",[미국],로렌스피시번 조판토리아노 키아누리브스 휴고위빙 글로리아포스터 캐리앤모스,릴리워쇼스키 라나워쇼스키,136.0,"티빙, 왓챠, 쿠팡플레이, 웨이브",False,5


In [28]:
netflix_data = data[data['provider'].str.contains('넷플릭스')]
watcha_data = data[data['provider'].str.contains('왓챠')]
wave_data = data[data['provider'].str.contains('웨이브')]
tiving_data = data[data['provider'].str.contains('티빙')]
disney_data = data[data['provider'].str.contains('디즈니+')]
seezn_data = data[data['provider'].str.contains('seezn')]
laftel_data = data[data['provider'].str.contains('라프텔')]
coupang_data = data[data['provider'].str.contains('쿠팡플레이')]

In [32]:
# 시사, 다큐멘터리 제거
data_tv = data[data['genre'].str.contains('시사', '다큐멘터리')==False]

In [33]:
# 드라마 예능 애니 개수
drama_count = {}
ent_count = {}
ani_count = {}
# 각 플랫폼 당 보유 컨텐츠 개수
for pf in platform:
  dr = 0
  en = 0
  an = 0
  for ki, gn, pv in zip(data_tv.kind, data_tv.genre, data_tv.provider):
    if ki == 'TV':
      if ('예능' in gn) and pf in pv:
        en += 1
      elif ('애니메이션' in gn) and pf in pv:
        an += 1
      else:
        if ('애니메이션' not in gn) and pf in pv:
          dr += 1
  ent_count[pf] = en
  ani_count[pf] = an
  drama_count[pf] = dr

tv_spec = pd.DataFrame({'drama':drama_count.values(), 'ent':ent_count.values(), 'ani':ani_count.values()}, index=platform)
tv_spec

,drama,ent,ani
넷플릭스,2199,321,752
웨이브,2762,608,743
티빙,1369,1188,781
쿠팡플레이,380,46,160
왓챠,2303,604,1112
디즈니+,489,25,206
seezn,289,107,10
라프텔,0,0,1285


In [39]:
# 2000년 기준 이전 컨텐츠가 많은 왓챠와 티빙 분류
before_2000_count = {}
after_2000_count = {}
# 각 플랫폼 당 보유 컨텐츠 개수
for pf in platform:
  b2 = 0
  a2 = 0
  for yr, pv in zip(data.year, data.provider):
    if (yr < 2000) and pf in pv:
      b2 += 1
    elif (yr >= 2000) and pf in pv:
      a2 += 1
  before_2000_count[pf] = b2
  after_2000_count[pf] = a2

year_2000 = pd.DataFrame({'before_2000':before_2000_count.values(), 'after_2000':after_2000_count.values()}, index=platform)
year_2000

,before_2000,after_2000
넷플릭스,205,6238
웨이브,1363,9058
티빙,470,9183
쿠팡플레이,67,1331
왓챠,1493,12383
디즈니+,278,1323
seezn,226,1526
라프텔,96,1605


In [ ]:
# 시각화를 위한 각 플랫폼 별 특성 json으로 변환